In [ ]:
# 라이브러리 import
!pip install tensorflow
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg19 import preprocess_input, decode_predictions
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, recall_score
import numpy as np
import os
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd "/content"
!mkdir img4
!unzip '/content/drive/MyDrive/Colab Notebooks/img4.zip' -d '/content/img4'

In [ ]:
# 데이터 디렉토리 설정
base_dir = '/content/img4'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

In [ ]:
# data generator
train_gen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,         # 이미지 굴절 비율
    brightness_range = (0.8,1.2), # 이미지 밝기
    horizontal_flip=True,    # 이미지 수평 반전 여부
)

img_gen = ImageDataGenerator(rescale=1./255)

train_set = img_gen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical')

val_set = img_gen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical',
    shuffle=False)

test_set = img_gen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical',
    shuffle=False)

Found 33067 images belonging to 5 classes.
Found 8268 images belonging to 5 classes.
Found 27996 images belonging to 5 classes.


In [ ]:
# VGG19 모델 설정 및 학습
vgg19 = VGG19(weights='imagenet', include_top=False)

for layer in vgg19.layers:
    layer.trainable = False

x = vgg19.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.7)(x)   # dropout 레이어
predictions = Dense(5, activation='softmax')(x)
model = Model(inputs=vgg19.input, outputs=predictions)

adam = Adam(lr=0.0001)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# 라벨과 언라벨 데이터 비율 설정
labeled_ratio = 0.1  # 라벨 데이터 비율
unlabeled_ratio = 0.9  # 언라벨 데이터 비율

In [ ]:
# 라벨 데이터 개수 계산
num_labeled_data = int(len(train_set) * labeled_ratio)
num_unlabeled_data = int(len(train_set) * unlabeled_ratio)

In [ ]:
# 라벨과 언라벨 데이터 분할
labeled_data_generator = train_gen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=num_labeled_data,
    class_mode='categorical'
)

unlabeled_data_generator = train_gen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=num_unlabeled_data,
    class_mode='categorical',
    shuffle=False
)


Found 33067 images belonging to 5 classes.
Found 33067 images belonging to 5 classes.


In [ ]:
# 라벨 데이터로 모델 학습
model.fit(
    labeled_data_generator,
    epochs=10,
    validation_data=val_set
)

Epoch 1/10
161/161 [==============================] - 493s 3s/step - loss: 1.5593 - accuracy: 0.3479 - val_loss: 1.2005 - val_accuracy: 0.6368
Epoch 2/10
161/161 [==============================] - 489s 3s/step - loss: 1.2200 - accuracy: 0.5147 - val_loss: 1.0693 - val_accuracy: 0.6586
Epoch 3/10
161/161 [==============================] - 489s 3s/step - loss: 1.1349 - accuracy: 0.5551 - val_loss: 1.0045 - val_accuracy: 0.6852
Epoch 4/10
161/161 [==============================] - 488s 3s/step - loss: 1.0892 - accuracy: 0.5766 - val_loss: 0.9593 - val_accuracy: 0.6911
Epoch 5/10
161/161 [==============================] - 488s 3s/step - loss: 1.0654 - accuracy: 0.5829 - val_loss: 0.9301 - val_accuracy: 0.6998
Epoch 6/10
161/161 [==============================] - 489s 3s/step - loss: 1.0491 - accuracy: 0.5883 - val_loss: 0.9089 - val_accuracy: 0.7017
Epoch 7/10
161/161 [==============================] - 489s 3s/step - loss: 1.0329 - accuracy: 0.5952 - val_loss: 0.8932 - val_accuracy: 0.7130

In [ ]:
# Pseudo Labeling을 위한 예측 결과 생성
test_set.reset()
pseudo_labels = model.predict(test_set)
pseudo_labels = np.argmax(pseudo_labels, axis=1)

1750/1750 [==============================] - 141s 81ms/step


In [ ]:
from tensorflow.keras.utils import to_categorical

# Pseudo Labeling된 unlabeled data를 기존 labeled data에 추가
labeled_data = np.concatenate((labeled_data_generator[0][0], unlabeled_data_generator[0][0]))
labeled_labels = np.concatenate((labeled_data_generator[0][1], to_categorical(pseudo_labels, num_classes=5)))

In [ ]:
# 모델 재설정 및 학습
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Pseudo Labeling된 데이터로 모델 재학습
model.fit(
    labeled_data[:num_labeled_data],
    labeled_labels[:num_labeled_data],
    epochs=10,
    validation_data=val_set
)

Epoch 1/10
7/7 [==============================] - 43s 7s/step - loss: 1.1248 - accuracy: 0.5146 - val_loss: 0.8626 - val_accuracy: 0.7155
Epoch 2/10
7/7 [==============================] - 43s 7s/step - loss: 1.0857 - accuracy: 0.5291 - val_loss: 0.8693 - val_accuracy: 0.7049
Epoch 3/10
7/7 [==============================] - 43s 7s/step - loss: 1.0783 - accuracy: 0.5583 - val_loss: 0.8648 - val_accuracy: 0.7111
Epoch 4/10
7/7 [==============================] - 43s 7s/step - loss: 1.1501 - accuracy: 0.5534 - val_loss: 0.8630 - val_accuracy: 0.6915
Epoch 5/10
7/7 [==============================] - 44s 7s/step - loss: 1.0821 - accuracy: 0.5631 - val_loss: 0.8698 - val_accuracy: 0.6769
Epoch 6/10
7/7 [==============================] - 44s 7s/step - loss: 1.0382 - accuracy: 0.5583 - val_loss: 0.8702 - val_accuracy: 0.7021
Epoch 7/10
7/7 [==============================] - 44s 7s/step - loss: 1.0661 - accuracy: 0.5680 - val_loss: 0.8685 - val_accuracy: 0.7088
Epoch 8/10
7/7 [==================

In [ ]:
# 모델 저장
model.save('vgg19_model.h5')

In [ ]:
import tensorflow as tf

from keras.utils import load_img, img_to_array, array_to_img
from PIL import Image
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import OneHotEncoder

images = []
labels = []

def load_image():
  global images, labels
  dir = "/content/img4/test/"
  class_dir = ['노균병','노균병유사','정상','흰가루병','흰가루병유사']
  for index in range(len(class_dir)):
    list_dir = dir+class_dir[index]+"/"
    for j in os.listdir(list_dir):
      if j == ".ipynb_checkpoints":
        continue
      img = load_img(list_dir+j)
      test_array = img_to_array(img)
      test_x = tf.expand_dims(img,0)
      images.append(test_x)
      labels.append(index)
load_image()
labels = np.array(labels)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

y = []
for i in range(len(images)):
  predictions = model.predict(images[i]/255)
  pred = np.argmax(predictions)
  y.append(pred)
class_names = list(test_set.class_indices.keys())
confusion_matrix = confusion_matrix(y, labels)
print(confusion_matrix)
print('Classification Report:')
print(classification_report(y, labels, target_names=class_names))

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
1/1 [==============================] - 0s 22ms/step
[[1262 1206  195   36    1]
 [2179 1952  896   47    2]
 [ 934 2214 7610   87   51]
 [ 723 1850  325 1250    3]
 [ 122  986 3236  812   17]]
Classification Report:
              precision    recall  f1-score   support

         노균병       0.24      0.47      0.32      2700
       노균병유사       0.24      0.38      0.29      5076
          정상       0.62      0.70      0.66     10896
        흰가루병       0.56      0.30      0.39      4151
      흰가루병유사       0.23      0.00      0.01      5173

    accuracy                           0.43     27996
   macro avg       0.38      0.37      0.33     27996
weighted avg       0.43      0.43      0.40     27996



In [ ]:
from sklearn import metrics

confusion_matrix = metrics.confusion_matrix(y, labels) #test_set
print(confusion_matrix)

[[1262 1206  195   36    1]
 [2179 1952  896   47    2]
 [ 934 2214 7610   87   51]
 [ 723 1850  325 1250    3]
 [ 122  986 3236  812   17]]
